### Genetic Algorithm 
- uses energy efficiency dataset https://archive.ics.uci.edu/ml/datasets/energy+efficiency#


We perform energy analysis using 12 different building shapes simulated in Ecotect. The buildings differ with respect to the glazing area, the glazing area distribution, and the orientation, amongst other parameters. We simulate various settings as functions of the afore-mentioned characteristics to obtain 768 building shapes. The dataset comprises 768 samples and 8 features, aiming to predict two real valued responses. It can also be used as a multi-class classification problem if the response is rounded to the nearest integer.


Attribute Information:

The dataset contains eight attributes (or features, denoted by X1...X8) and two responses (or outcomes, denoted by y1 and y2). The aim is to use the eight features to predict each of the two responses. 

Specifically: 
- X1	Relative Compactness 
- X2	Surface Area 
- X3	Wall Area 
- X4	Roof Area 
- X5	Overall Height 
- X6	Orientation 
- X7	Glazing Area 
- X8	Glazing Area Distribution 
- y1	Heating Load 
- y2	Cooling Load



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn import cross_validation
from sklearn import preprocessing
from sklearn import svm

### Get the dataset
- df.sample(frac=1) shuffles the data 

In [ ]:
df = pd.read_csv("energy_efficiency.csv")
df = df.sample(frac=1)
rows = len(df)
cols = len(df.keys())
print(rows, cols)
df.head()

### X6 and X8 are categorical variables

In [ ]:
X = pd.get_dummies(df.iloc[:,0:(cols-2)], columns=['X6', 'X8']).values
y = df.iloc[:, (cols-2):(cols-1)].values
print(X[0])
print(y[0])

### Scale the data with minmax scaler

In [ ]:
mms = preprocessing.MinMaxScaler()
X = mms.fit_transform(X)
print(X[0])

### Set genetic algorithm parameters
- M : number of generations
- N : population size
- Pc: probability of crossover
- Pm: probability of mutation
- l : string size
- k : tournament selection contestants

### **since we are performing a GA for SVM, the parameters to tune are c and gamma**

#### SVM hyperparameters
- kernel: poly, rbf, linear
- poly: degree of polynomial
- c: controls the tradeoff between low error and maximizing the norm of the weight
- gamma: determines strength of training sample with gaussian kernel

In [ ]:
#probability of crossover
Pc = .95 #these do not need to sum to 1
#probability of mutation
Pm = .08
#size of population
pop = 100
#number of generations
gen = 50
l = 24 # first 12 are for c, second 12 are for gamma

In [ ]:
# the chromosome
xy = np.random.choice([0,1], size=(l,), p=[.5, .5])
print(xy)

### (1) Create a random population of chromosomes (potential solutions)

In [ ]:
population = np.empty((0, len(xy)))
for i in range(gen):
    random.shuffle(xy)
    population = np.vstack((population, xy))

In [ ]:
print(population[0:5])

### (2) Calculate precision of the chromosomes
- Range (a,b)
- length, l
- precision (b-a)/(2^l-1)
- encode (literal base 2 encoding)
- decode (sum(bit*2^i)*precision + a

In [ ]:
# parameter c, see precision formula
ac = 10
bc = 1000
lc = len(xy)/2
pc = (bc - ac)/((2**lc)-1)

In [ ]:
# parameter gamma, see precision formula
ag = .05
bg = .99
lg = len(xy)/2
pg = (bg - ag)/((2**lg)-1)

### Decode the chromosomes
- returns a real value in the specified range for the parameter

In [ ]:
def decode(xy, index, precision, lowerBound):
    """Decodes a chromosome into a real value"""
    power = 0 #binary powers
    sum = 0
    for i in range(len(xy)//2):
        val = xy[index]*(2**power)
        sum += val
        index -= 1
        power += 1
    return (sum * precision) + lowerBound

In [ ]:
#index of the start of the c parameter from SVM 
cIndex = -1 # start at the end of the chromosome
#index of the start of the gamma parameter from SVM
gIndex = (l//2) + 1 #start in the middle of the chromosome
c1 = decode(xy, cIndex, pc, ac)
g1 = decode(xy, gIndex, pg, ag)
print("c bounds: ({},{})\tcvalue: {}".format(ac,bc,c1))
print("g bounds: ({},{})\tgvalue: {}".format(ag,bg,g1))


### Perform the algorithm

- keep track of children and mutations 

In [ ]:
def kfold(choice, num_folds):
    c = decode(choice, cIndex, pc, ac)
    g = decode(choice, gIndex, pg, ag)
    total = 0
    kf = cross_validation.KFold(len(X),n_folds=num_folds)

    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = svm.SVR(kernel='rbf',C=c, gamma=g)
        model.fit(X_train, y_train)
        acc = model.score(X_test, y_test)
        err = 1-acc
        total += err
    #need the total error
    total = total/num_folds
    return total

def parentSelection(population, l):
    """
    returns two parents from a population given the chromosome size l
    """
    parents = np.zeros((0, l))
    for k in range(2):
        #get some random samples to pick good parents
        candidates = population[np.random.choice(len(population), 3, replace=False)]

        #the best selection for a parent
        selection = -1
        score = 10000000

        for choice in candidates:
            #c and gamma values for SVR
            total = kfold(choice, num_folds)
            #print(total)
            if (total < score):
                score = total
                selection = choice

        #print(selection, score)
        parents = np.vstack((parents, selection))
    return parents

def crossover(parents):
    children = np.copy(parents)

    #perform crossover where Pc is the probability that crossover takes place
    if(np.random.rand() < Pc):
        #get 2 random indicies for the crossover portion
        indices = np.random.choice(l,2,replace=False)
        #ensure first index is smaller than the second
        if(indices[0] > indices[1]):
            temp = indices[0]
            indices[0] = indices[1]
            indices[1] = temp
        #print(indices)

        children[0][indices[0]:indices[1]] = parents[1][indices[0]:indices[1]]
        children[1][indices[0]:indices[1]] = parents[0][indices[0]:indices[1]]
    return children

def mutate(children):
    for child in children:
        for i in range(0, len(child)):
            if(np.random.rand() < Pm):
                #print(i)
                if(child[i] == 0):
                    child[i] = 1
                else:
                    child[i] = 0


In [ ]:
bestC = []
worstC = []
bestG = []
worstG = []

ofg = np.empty((0, len(xy)+2))
ofgf = []

mgm0 = np.empty((0, len(xy)+1))
mgm1 = np.empty((0, len(xy)+1))

mgm00 = np.empty((0, len(xy)+2))
mgm11 = np.empty((0, len(xy)+2))

mgm000 = np.empty((0, len(xy)+2))
mgm111 = np.empty((0, len(xy)+2))

In [ ]:
#index of the start of the c parameter from SVM 
cIndex = -1 # start at the end of the chromosome
#index of the start of the gamma parameter from SVM
gIndex = (l//2) + 1 #start in the middle of the chromosome

for i in range(1):
    print("generation: ", i)
    
    newPopulation = np.empty((0, len(xy)))
    
    agc0 = np.empty((0, len(xy)+1))
    agc1 = np.empty((0, len(xy)+1))
    
    mgc0 = []
    mgc1 = []
    
    bestgc = np.empty((0, len(xy)+1))
    fbgc = []
    fwgc = []
    
    num_folds = 3
    
    
    for j in range(3): #pop//2
        print("family: ", j)
        
        parents = parentSelection(population, l)
        children = crossover(parents)
        mutate(children)
        fitness = []
        for child in children:
            fitness.append(kfold(child, num_folds))
        print("fitness of child 1 in family {} is {}".format(j, fitness[0]))
        print("fitness of child 2 in family {} is {}".format(j, fitness[1]))
        
        temp0 = np.hstack((fitness[0], children[0]))
        temp1 = np.hstack((fitness[1], children[1]))

        agc0 = np.vstack((agc0, temp0))
        agc1 = np.vstack((agc1, temp1))
        bestgc = np.vstack((agc0,agc1))
        #print(bestgc)
        newPopulation = np.vstack((newPopulation, children[0], children[1]))
        
        r0 = min(agc0[:,:1])
        for i in range(0, len(agc0)):
            if(agc0[i,:1] == r0):
                mgc0 = agc0[i,:]
        #print("mgc0: ", mgc0)
        
        r1 = min(agc1[:,:1])
        for i in range(0, len(agc1)):
            if(agc1[i,:1] == r1):
                mgc1 = agc1[i,:]
        #print("mgc1: ", mgc1)
        
    best = min(bestgc[:,:1])
    for i in range(0, len(bestgc)):
        if(bestgc[i,:1] == best):
            fbgc = bestgc[i,:]

    worst = max(bestgc[:,:1])
    for i in range(0, len(bestgc)):
        if(bestgc[i,:1] == worst):
            fwgc = bestgc[i,:]
            
    #print(fbgc)
    #print(fwgc)
    #print(fwgc[1:])
    #print(newPopulation)
    
    for i in range(0, len(newPopulation)):
        #print(newPopulation[i])
        #print(fwgc[1:])
        #print()
        if(np.array_equal(newPopulation[i], fwgc[1:])):
            #print("found! : ", newPopulation[i])
            newPopulation[i] = fbgc[1:]
    #print(newPopulation)
    population = newPopulation
    print(mgc0.shape)
    mgm0 = np.vstack((mgm0, mgc0))
    mgm1 = np.vstack((mgm1, mgc1))
    print(mgm0.shape)
    
    #add generation number
    mgm00 = np.insert(mgm0, 0, i)
    mgm11 = np.insert(mgm1, 0, i)
    print(mgm00.shape)
    print(mgm000.shape)
    
    mgm000 = np.vstack((mgm000, mgm00))
    mgm111 = np.vstack((mgm111, mgm00))
    
ofg = np.vstack((mgm000, mgm111))
print(ofg)

In [ ]:
best = min(ofg[:,:1])
for i in range(0, len(bestgc)):
    if(ofg[i,:1] == best):
        ofgf = ofg[i,:]
print("Min for all generations: ", ofgf)